In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
FOLDER = "MIMIC-III Demo"

In [ ]:
# Load patients data
patients = pd.read_csv(f"{FOLDER}/PATIENTS.csv")

In [ ]:
# Add age for each patient
def calculate_age(dob, dod):
    dob_date = datetime.strptime(dob, "%Y-%m-%d %H:%M:%S")
    dod_date = datetime.strptime(dod, "%Y-%m-%d %H:%M:%S")
    age = (dod_date - dob_date).days // 365
    return age

patients["age"] = patients.apply(lambda row: calculate_age(row["dob"], row["dod"]), axis=1)

In [ ]:
# Check how many patients have invalid age
# (don't want to full drop these yet, but will need to for specific visualizations)
(patients["age"] > 120).sum()

In [ ]:
# Load admissions data
admissions = pd.read_csv(f"{FOLDER}/ADMISSIONS.csv")

In [ ]:
# Get length of stay (in days)
def calculate_length_of_stay(admittime, dischtime):
    admit_date = datetime.strptime(admittime, "%Y-%m-%d %H:%M:%S")
    disch_date = datetime.strptime(dischtime, "%Y-%m-%d %H:%M:%S")
    return (disch_date - admit_date).days

admissions["length_of_stay"] = admissions.apply(
    lambda row: calculate_length_of_stay(row["admittime"], row["dischtime"]), 
    axis=1
)

In [ ]:
# Get diagnosis data
diagnoses = pd.read_csv(f"{FOLDER}/DIAGNOSES_ICD.csv")

In [ ]:
# Get ICD9 lookup
icd_lookup = pd.read_csv(f"{FOLDER}/D_ICD_DIAGNOSES.csv")

In [ ]:
# Join to get ICD descriptions
diagnoses = diagnoses.merge(icd_lookup[["icd9_code", "short_title"]], on="icd9_code", how="left")

In [ ]:
# Rename for clarity
diagnoses.rename(columns={"short_title": "icd9_desc"}, inplace=True)

In [ ]:
# Check for diagnosis codes that don't appear in the lookup table
# (could try to address this, but not doing that right now)
diagnoses[diagnoses["icd9_desc"].isna()]["icd9_code"].value_counts(ascending=False)

In [ ]:
# Get primary diagnosis for each patient-admission combo
diagnoses_seq1 = diagnoses.query("seq_num == 1")

# (1) Diagnoses for readmitted patients

In [ ]:
# Get patients with multiple admissions
multiple_admissions = admissions.groupby("subject_id").filter(lambda x: x.shape[0] > 1).sort_values("subject_id")

In [ ]:
multiple_admissions.shape

In [ ]:
# Join to get top diagnosis for each admission
multiple_admissions = multiple_admissions.merge(
    diagnoses_seq1[["subject_id", "hadm_id", "icd9_code", "icd9_desc"]],
    on=["subject_id", "hadm_id"],
    how="inner"
)

In [ ]:
# Add column for sequence of admissions
multiple_admissions["admit_num"] = multiple_admissions.sort_values(["subject_id", "admittime"]).groupby("subject_id").cumcount() + 1

# For simplicity, only consider first two admissions
multiple_admissions = multiple_admissions.query("admit_num < 3")

# Reshape for easier comparison of first and second diagnosis
plot_df = multiple_admissions[["subject_id", "admit_num", "icd9_desc"]].pivot(
    index="subject_id",
    columns="admit_num",
    values="icd9_desc"
)

In [ ]:
# Rename columns
plot_df.columns = ["code_1", "code_2"]

In [ ]:
# Add flag for whether diagnosis is same for 1 and 2
# (i.e., primary reason for both visits was the same)
plot_df["same_code"] = (plot_df["code_1"] == plot_df["code_2"])

In [ ]:
# Check how often diagnosis is the same
plot_df["same_code"].value_counts()

In [ ]:
plot_df

In [ ]:
# Get count of occurrences across code 1 and 2
pd.concat([plot_df["code_1"], plot_df["code_2"]]).value_counts()

In [ ]:
# Plot graph of different combinations
import networkx as nx

edges = plot_df[['code_1', 'code_2']].values.tolist()
G = nx.Graph()
G.add_edges_from(edges)
pos = nx.random_layout(G, seed=42)
nx.draw(G, pos, with_labels=True)
plt.show()

# 2) Diagnoses and prescriptions

In [ ]:
# Load prescription data
prescriptions = pd.read_csv(f"{FOLDER}/PRESCRIPTIONS.csv")

In [ ]:
# Get unique drugs for each patient-admission combo
drugs_by_visit = prescriptions[["subject_id", "hadm_id", "drug"]].drop_duplicates()

In [ ]:
# Join with diagnosis data
drugs_and_diagnoses = diagnoses.merge(
    drugs_by_visit,
    on=["subject_id", "hadm_id"],
    how="inner"
)

In [ ]:
# To make plot manageable, filter to top 10 most frequent drugs and diagnoses
top10_drugs = drugs_by_visit["drug"].value_counts(ascending=False).index.tolist()[0:10]
top10_diagnoses = diagnoses["icd9_desc"].value_counts(ascending=False).index.tolist()[0:10]
drugs_and_diagnoses_filtered = drugs_and_diagnoses[
    (drugs_and_diagnoses["drug"].isin(top10_drugs))
    & (drugs_and_diagnoses["icd9_desc"].isin(top10_diagnoses))
]

In [ ]:
# Create heatmap
plot_df = drugs_and_diagnoses_filtered.groupby(["icd9_desc", "drug"]).size().unstack().fillna(0)

# Create a heatmap
plt.figure(figsize=(12, 8))
# sns.heatmap(plot_df, annot=True, fmt='g', cmap='YlGnBu')
sns.heatmap(plot_df, annot=True, cmap="Blues")
plt.title("Diagnosis / Drug Co-Occurrence")
plt.xlabel("Drug")
plt.ylabel("Diagnosis")
plt.xticks(rotation=45, ha="right")

# Show the plot
plt.show()

In [ ]:
# Average co-occurrence
drugs_and_diagnoses_filtered.groupby(["icd9_desc", "drug"]).size().mean()

# 3) Time to death after discharge

In [ ]:
# Get latest discharge for patients that didn't die in hosptial
latest_disch = admissions.query("hospital_expire_flag != 1")[["subject_id", "hadm_id", "dischtime"]]
latest_disch = latest_disch.sort_values(["subject_id", "dischtime"], ascending=False).drop_duplicates("subject_id", keep="first")

In [ ]:
# Get discharge datetime and death datetime in single df
# and calculate time to death after discharge
def calc_disch_to_dod(disch, dod):
    disch_date = datetime.strptime(disch, "%Y-%m-%d %H:%M:%S")
    death_date = datetime.strptime(dod, "%Y-%m-%d %H:%M:%S")
    return (death_date - disch_date).days

plot_df = pd.merge(
    patients[["subject_id", "dod", "age"]].query("age < 120"),
    latest_disch,
    on="subject_id",
    how="inner"
)

plot_df["disch_to_dod"] = plot_df.apply(
    lambda row: calc_disch_to_dod(row["dischtime"], row["dod"]), 
    axis=1
)

In [ ]:
# Add avg length of stay from admissions data
avg_length_of_stay = admissions.groupby("subject_id")["length_of_stay"].mean().reset_index()
plot_df = plot_df.merge(avg_length_of_stay, on="subject_id", how="inner")

sns.lmplot(x="length_of_stay", y="disch_to_dod", data=plot_df, ci=None)
plt.show()

# 4) Distribution of Heart Rate

In [ ]:
# Load chart events data
chart_events = pd.read_csv(f"{FOLDER}/CHARTEVENTS.csv", low_memory=False)

In [ ]:
# Load items lookup table
items_lookup = pd.read_csv(f"{FOLDER}/D_ITEMS.csv", low_memory=False)

In [ ]:
# Get relevant item IDs for heart rate
# NOTE: one for carevue, one for metavision
heartrate_ids = items_lookup.query("label == 'Heart Rate'")["itemid"].tolist()

In [ ]:
# Get heart rate measurements only
heartrates = chart_events[chart_events["itemid"].isin(heartrate_ids)]

In [ ]:
# Looks like 2 patients don't have heart rate measurements
heartrates["subject_id"].nunique()

In [ ]:
# Drop null or 0 heart rates
# (0 heart rate might be valid for patients that died in hospital, but it will
#  skew the averages)
heartrates = heartrates[(~heartrates["valuenum"].isna()) & (heartrates["valuenum"] > 0)]

In [ ]:
# Get average heart rate per patient
# NOTE: grouping across admissions for patients with multiple admissions
avg_heartrate = heartrates.groupby("subject_id")["valuenum"].mean().reset_index()

In [ ]:
# Plot histogram by marital status

# Get marital status from admissions data
marital_status = admissions[["subject_id", "marital_status"]].drop_duplicates()

# CLean up marital status by creating "other" category
marital_status.loc[~marital_status["marital_status"].isin(["SINGLE", "DIVORCED", "MARRIED", "WIDOWED"]), "marital_status"] = "OTHER"

# Get marital status and average heartrate in same df
plot_df = pd.merge(marital_status, avg_heartrate, on="subject_id", how="inner")

# Set column names
plot_df.columns = ["subject_id", "marital_status", "bpm"]

g = sns.FacetGrid(plot_df, col='marital_status')
g.map(plt.hist, "bpm")
g.add_legend()
plt.show()

In [ ]:
# Get average of averages within each group
plot_df.groupby("marital_status")["bpm"].mean().sort_values().reset_index()

# 5) Show all chart events for single patient

In [ ]:
# Pick patient ID
patient_of_interest = patients["subject_id"][0]

In [ ]:
# Get all chart events for this patient
single_patient_chart_events = chart_events.query(f"subject_id == {patient_of_interest}")

In [ ]:
# Join to get labels for events
single_patient_chart_events = single_patient_chart_events.merge(
    items_lookup[["itemid", "label"]],
    on="itemid",
    how="inner"
)

In [ ]:
# Filter to events that have a numeric value
single_patient_chart_events = single_patient_chart_events[~single_patient_chart_events["valuenum"].isna()]

In [ ]:
# Get events that occurred more than 5 times
# and filter to those events
temp_freq = single_patient_chart_events["label"].value_counts()
frequent_events = temp_freq[temp_freq >= 5].index.tolist()
single_patient_chart_events = single_patient_chart_events[single_patient_chart_events["label"].isin(frequent_events)]

In [ ]:
# Get events that have some variation (for more interesting plots)
single_patient_chart_events = single_patient_chart_events.groupby("label").filter(lambda x: x["valuenum"].mean() != x["valuenum"].max())

In [ ]:
# Don't need temperature in both Celsius and Fahrenheit
single_patient_chart_events = single_patient_chart_events.query("label != 'Temperature C (calc)'")

In [ ]:
# Plot all chart events over time

plot_df = single_patient_chart_events[["label", "valuenum"]].copy()

# Convert to datetime
plot_df["charttime"] = pd.to_datetime(single_patient_chart_events['charttime'])

g = sns.FacetGrid(single_patient_chart_events, col='label', col_wrap=5, sharex=True)
g.map(plt.plot, "charttime", "valuenum")
g.map(plt.xticks, rotation=90)
g.add_legend()
plt.show()

# BONUS: analyze patient with 15 different admissions

In [ ]:
# Get patient ID
bad_luck_patient = admissions.groupby('subject_id').size().sort_values(ascending=False).index[0]

In [ ]:
# Patient didn't die in hospital (so maybe luck wasn't the worst?)
patients.query(f"subject_id == {bad_luck_patient}")["dod_hosp"]

In [ ]:
# Get admissions for this patient
admissions_filter = admissions.query(f"subject_id == {bad_luck_patient}")

In [ ]:
# Get primary diagnosis for each admission
plot_df = pd.merge(admissions_filter, diagnoses_seq1, on=["subject_id", "hadm_id"], how="left")

# Convert admit time to datetime and extract day
plot_df["admit_date"] = pd.to_datetime(plot_df["admittime"]).dt.date

# Plot different types of admissions over time
plt.scatter(plot_df["admit_date"], plot_df["icd9_desc"])
plt.xticks(rotation=45)
plt.show()